In [1]:
import pandas as pd
import numpy as np
import requests
import bs4
from bs4 import BeautifulSoup
import re

In [2]:
data = pd.read_excel('/Users/riyamhatre/Desktop/Online Retail.xlsx')

In [3]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
# Scraper to get the list of holidays for each country 
def holidays(country, year):
    def get_info(country, year):
        url = 'https://www.timeanddate.com/holidays/{}/{}'.format(country, year)
        request = requests.get(url).text
        return request
    soup = bs4.BeautifulSoup(get_info(country, year), features="html.parser")
    holiday_names = [a_tag.text for a_tag in soup.find_all('a')]

    holiday_names = holiday_names[holiday_names.index("DEC")+1:holiday_names.index('let us know')]
    
    # Loop through each row to extract holiday types
    rows = soup.find_all('tr')
    holiday_type = []
    for row in rows:
        tds = row.find_all('td')
        if len(tds) >= 3:  
            holiday_type.append(tds[2].text.strip()) 
    
    elements = soup.find_all(class_="nw")
    count = 0
    date = []
    day = []
    for i in elements: 
        if count%2 == 0:
            date.append(i.text)
        else:
            day.append(i.text)
        count +=1
    holidays = pd.DataFrame({"Date": date,"Day": day,"Holiday": holiday_names, "Holiday Type" : holiday_type})
    #only the most important holidays will be considered
    holiday_type = ['national holiday, hebrew', 'bank holiday', 'federal holiday', 'national holiday', 'public holiday']
    holidays['Holiday Type'] = holidays['Holiday Type'].str.lower()
    
    holidays = holidays[holidays['Holiday Type'].isin(holiday_type)][['Date','Holiday']].rename(columns = {"Holiday": country})
    
    return holidays

In [6]:
month_dict = {
    'Jan': 'January',
    'Feb': 'February',
    'Mar': 'March',
    'Apr': 'April',
    'May': 'May',
    'Jun': 'June',
    'Jul': 'July',
    'Aug': 'August',
    'Sep': 'September',
    'Oct': 'October',
    'Nov': 'November',
    'Dec': 'December'
}
# create the table with all the holiday data and add it to the master data
def table_creation(year, start, end):
    nonexistent = ["channel islands", "czech republic", 'bahrain', 'unspecified','rsa', 'european community'] #no holiday data for these 
    countries = data['Country'].unique()
    all_days = pd.DataFrame()
    
    date_range = pd.date_range(start=start, end=end)

    # Create the DataFrame
    alldays = pd.DataFrame({
        'Date': date_range.strftime('%b %-d'),  
        'Day of Week': date_range.strftime('%A')  
    })
    places = list(map(lambda x: x.lower(), data['Country'].unique().tolist()))
    places_dict = {'united kingdom': 'uk', 'eire':'ireland','united arab emirates': 'united-arab-emirates', 'saudi arabia':'saudi-arabia', 'hong kong': 'china', 'usa': 'us'}
    places = [places_dict.get(item, item) for item in places]
    for i in range(len(places)):
        if places[i] in nonexistent:
            continue
        else:
            temp_df = holidays(places[i],year) # list of holidays for that country
            temp_df = temp_df.rename(columns = {places[i]: countries[i]})
            temp_df = pd.merge(temp_df, alldays, on = "Date", how = 'right')
            all_days = pd.concat([all_days, temp_df], axis=1).drop(columns = {"Date", 'Day of Week'}) # add to df
            
    # Put all the data together
    all_days = all_days.iloc[:-3] # last 3 entries are irrelevant
    all_days['Date'] = alldays['Date']
    all_days['Day of Week'] = alldays['Day of Week']
    
    all_days['Month'] = all_days["Date"].str.split().apply(lambda x: x[0])
    all_days['Day'] = all_days["Date"].str.split().apply(lambda x: x[1])
    all_days["Month"] = all_days['Month'].replace(month_dict)
    
    all_days_cols = list(all_days.columns[:-4])
    data['Month'] = data['InvoiceDate'].dt.month_name()
    
    year_table = data[data['InvoiceDate'].dt.year== int(year)]
    year_table['CombinedDate'] = year_table["Month"] + " " +year_table['InvoiceDate'].apply(lambda x: x.strftime("%-d"))
    all_days['CombinedDate'] = all_days['Month'] + " " + all_days['Day']
    
    holidays_long = all_days.melt(id_vars=['CombinedDate'], var_name='Country', value_name='Holiday')

    merged_df = pd.merge(year_table, holidays_long, on=['CombinedDate', 'Country'], how='left')
    
    return merged_df

The dataset only goes from 2010 to 2011, so we just need to run the table assembler twice.

In [7]:
holidays_2011 = table_creation('2011',start="2011-01-01", end="2011-12-31" )

/var/folders/vj/15h0623d3z7d6s8rzlr0t4b80000gn/T/ipykernel_5953/4014935788.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_table['CombinedDate'] = year_table["Month"] + " " +year_table['InvoiceDate'].apply(lambda x: x.strftime("%-d"))


In [8]:
holidays_2011.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,CombinedDate,Holiday
0,539993,22386,JUMBO BAG PINK POLKADOT,10,2011-01-04 10:00:00,1.95,13313.0,United Kingdom,January,January 4,NaN
1,539993,21499,BLUE POLKADOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom,January,January 4,NaN
2,539993,21498,RED RETROSPOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom,January,January 4,NaN
3,539993,22379,RECYCLING BAG RETROSPOT,5,2011-01-04 10:00:00,2.10,13313.0,United Kingdom,January,January 4,NaN
4,539993,20718,RED RETROSPOT SHOPPER BAG,10,2011-01-04 10:00:00,1.25,13313.0,United Kingdom,January,January 4,NaN


In [9]:
holidays_2010 = table_creation('2010',start="2010-01-01", end="2010-12-31" )

/var/folders/vj/15h0623d3z7d6s8rzlr0t4b80000gn/T/ipykernel_5953/4014935788.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_table['CombinedDate'] = year_table["Month"] + " " +year_table['InvoiceDate'].apply(lambda x: x.strftime("%-d"))


In [10]:
holidays_2010.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,CombinedDate,Holiday
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,December,December 1,NaN
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,December 1,NaN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,December,December 1,NaN
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,December 1,NaN
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,December 1,NaN


In [11]:
# from pathlib import Path
# filepath = Path('/Users/riyamhatre/Desktop/holidays_2010.csv')
# filepath.parent.mkdir(parents=True, exist_ok = True)
# holidays_2010.to_csv(filepath)

In [12]:
# from pathlib import Path
# filepath = Path('/Users/riyamhatre/Desktop/holidays_2011.csv')
# filepath.parent.mkdir(parents=True, exist_ok = True)
# holidays_2011.to_csv(filepath)